<a href="https://colab.research.google.com/github/eli-osherovich/mnist-demo/blob/main/mnist_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras.datasets import mnist, fashion_mnist

In [2]:
!nvidia-smi -L

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-3f01f1de-db72-d54f-7376-7bb1cfa64352)


In [3]:
tf.config.optimizer.set_jit(True)

In [4]:
# the data, split between train and test sets
dataset = fashion_mnist
dataset = mnist


(x_train, y_train), (x_test, y_test) = dataset.load_data()

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

x_train = x_train.astype(np.float32)/255
x_test = x_test.astype(np.float32)/255
y_train = y_train.astype(np.int)
y_test = y_test.astype(np.int)

print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (60000, 28, 28, 1)
x_test shape: (10000, 28, 28, 1)


In [5]:
def gen_model(activation="relu"):
  s = tf.keras.Input(shape=(28, 28, 1))
  h = tf.keras.layers.Flatten()(s)
  h = tf.keras.layers.Dropout(0.1)(h)

  h = tf.keras.layers.Dense(128, activation=activation)(h)
  h = tf.keras.layers.Dropout(0.4)(h)

  h = tf.keras.layers.Dense(128, activation=activation)(h)
  
  out = tf.keras.layers.Dense(10)(h)
  return tf.keras.Model(inputs=s, outputs=out)


def compile_model(model, lr=0.001):
  model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=keras.optimizers.Adam(learning_rate=lr),
                metrics=['accuracy'])
  

early_stop = tf.keras.callbacks.EarlyStopping(
      monitor='val_accuracy',
      patience=10,
      restore_best_weights=True,
)

In [6]:
%%time
model = gen_model()
compile_model(model)
model.fit(x_train, y_train,
          batch_size=128,
          epochs=100,
          verbose=1,
          callbacks=[early_stop],
          validation_data=(x_test, y_test),
          validation_batch_size=2048)


Epoch 1/100
469/469 [==============================] - 2s 4ms/step - loss: 0.4656 - accuracy: 0.8594 - val_loss: 0.1774 - val_accuracy: 0.9464
Epoch 2/100
469/469 [==============================] - 3s 5ms/step - loss: 0.2249 - accuracy: 0.9332 - val_loss: 0.1227 - val_accuracy: 0.9621
Epoch 3/100
469/469 [==============================] - 1s 3ms/step - loss: 0.1828 - accuracy: 0.9449 - val_loss: 0.1074 - val_accuracy: 0.9671
Epoch 4/100
469/469 [==============================] - 1s 3ms/step - loss: 0.1581 - accuracy: 0.9525 - val_loss: 0.0909 - val_accuracy: 0.9708
Epoch 5/100
469/469 [==============================] - 1s 3ms/step - loss: 0.1429 - accuracy: 0.9560 - val_loss: 0.0916 - val_accuracy: 0.9726
Epoch 6/100
469/469 [==============================] - 1s 3ms/step - loss: 0.1318 - accuracy: 0.9588 - val_loss: 0.0779 - val_accuracy: 0.9761
Epoch 7/100
469/469 [==============================] - 1s 3ms/step - loss: 0.1250 - accuracy: 0.9603 - val_loss: 0.0787 - val_accuracy: 0.9754

In [7]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Test loss: 0.06524636596441269
Test accuracy: 0.9841000437736511
